In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, MetaData, Table, Column, ForeignKey, String, Integer, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker


In [2]:
#https://www.sqlitetutorial.net/sqlite-python/creating-tables/
# Methods to handle database
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn
    


def load_csv_to_db(conn,csv_filename,table_name):
    df = pd.read_csv(csv_filename)
    df.to_sql(table_name, conn, if_exists='append', index = False)



In [4]:

metadata_obj = MetaData()
Ratings = Table(
    "Ratings",
    metadata_obj,
    Column("ratingId", Integer, primary_key=True),
    Column("userId", Integer),
    Column("movieId",Integer, ForeignKey("Movies.movieId")),
    Column("rating", Float),
    Column("timestamp", DateTime),
)

Movies = Table(
    "Movies",
    metadata_obj,
    Column("movieId", Integer, primary_key=True),
    Column("title", String),
    Column("genres", String)
)


In [15]:
Base = declarative_base()

# Sqlalchemy Tables:
class Rating(Base):
    __tablename__ = "Ratings"
    ratingId = Column("ratingId", Integer, primary_key=True)
    userId = Column("userId", Integer)
    movieId =Column(Integer, ForeignKey("Movies.movieId"))
    rating = Column("rating", Float)
    timestamp = Column("timestamp", DateTime)

    def __init__(self, userId,movieId,rating,timestamp):
        self.userId = userId
        self.movieId = movieId
        self.rating = rating
        self.timestamp = timestamp
    def __repr__(self):
        return f"{self.userId}, {self.movieId}, {self.rating}"



class Movie(Base):
    __tablename__ = "Movies"
    movieId = Column("movieId", Integer, primary_key=True)
    title = Column("title", String)
    genres =Column("genres", String)

    def __init__(self, movieId, title, genre):
        self.movieId = movieId
        self.title = title
        self.genre =genre
    def __repr__(self):
        return f"{self.movieId}, {self.title}, {self.genre}"

class Tag(Base):
    __tablename__ = "Tags"
    userId = Column("userId", Integer)
    tagId = Column("tagId", Integer, primary_key=True)
    movieId = Column(Integer, ForeignKey("Movies.movieId"))
    tag = Column("tag", String)
    timestamp = Column("timestamp", DateTime)
    def __init__(self, tagId, movieId, tag,timestamp, userId):
        self.movieId = movieId
        self.tagId = tagId
        self.tag =tag
        self.timestamp = timestamp
        self.userId = userId
    def __repr__(self):
        return f"{self.tag}, {self.movieId}"

class Score(Base):
    __tablename__ = "Genome_Scores"
    scoreId = Column("scoreId", Integer, primary_key=True)
    movieId = Column(Integer, ForeignKey("Movies.movieId"))
    tagId = Column("tagId", Integer)
    relevance = Column("relevance", Float)
    def __init__(self, tagId, movieId, scoreId,relevance):
        self.movieId = movieId
        self.tagId = tagId
        self.scoreId =scoreId
        self.relevance = relevance
    def __repr__(self):
        return f"{self.movieId}, {self.scoreId}, {self.relevance}"

class Genome_Tag(Base):
    __tablename__ = "Genome_Tags"
    tagId = Column( "tagId",Integer, primary_key=True)
    tag = Column("tag", String)
    def __init__(self, tagId, tag):
        self.tagId = tagId
        self.tag = tag
    def __repr__(self):
        return f"{self.tag}"

class Link(Base):
    __tablename__ = "Links"
    linkId = Column("linkId", Integer, primary_key=True)
    movieId = Column(Integer, ForeignKey("Movies.movieId"))
    imdbId = Column("imdbId", Integer)
    tmdbId = Column("tmdbId", Float)
    def __init__(self, imdbId, movieId, tmdbId,relevance):
        self.movieId = movieId
        self.imdbId = imdbId
        self.tmdbId =tmdbId
    def __repr__(self):
        return f"{self.movieId}, {self.imdbId}, {self.tmdbId}"





In [16]:

engine = create_engine("sqlite:///Database/movielens.db")
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()

In [17]:
#connect to database and import als csv files as tables
from sqlalchemy.types import Integer, String, DateTime
connection=create_connection("Database/movielens.db")
load_csv_to_db(connection, "movielens/tags.csv", "Tags")
load_csv_to_db(connection, "movielens/ratings.csv", "Ratings")
load_csv_to_db(connection, "movielens/genome-scores.csv", "Genome_Scores")
load_csv_to_db(connection, "movielens/genome-tags.csv", "Genome_Tags")
load_csv_to_db(connection, "movielens/links.csv", "Links")
load_csv_to_db(connection, "movielens/movies.csv","Movies")


2.6.0


OperationalError: table Tags has no column named userId

In [39]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor = connection.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('Tags',), ('Ratings',), ('Genome_Scores',), ('Genome_Tags',), ('Links',), ('Movies',)]


In [6]:
db = connection.cursor()
rows = db.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]

def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

for table in tables:
    print("table: " + table)
    rows = db.execute("PRAGMA table_info({})".format(sql_identifier(table)))
    print(rows.fetchall())
    rows = db.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    print(rows.fetchall())

table: Movies
[(0, 'movieId', 'INTEGER', 1, None, 1), (1, 'title', 'VARCHAR', 0, None, 0), (2, 'genres', 'VARCHAR', 0, None, 0)]
[]
table: Ratings
[(0, 'ratingId', 'INTEGER', 1, None, 1), (1, 'userId', 'INTEGER', 0, None, 0), (2, 'movieId', 'INTEGER', 0, None, 0), (3, 'rating', 'FLOAT', 0, None, 0), (4, 'timestamp', 'DATETIME', 0, None, 0)]
[(0, 0, 'Movies', 'movieId', 'movieId', 'NO ACTION', 'NO ACTION', 'NONE')]
